<a href="https://colab.research.google.com/github/adrien50/humanactivityrecognition/blob/main/imagerecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Recognition (Classification)

Image recognition refers to the task of inputting an image into a neural network and having it output some kind of label for that image. The label that the network outputs will correspond to a pre-defined class. There can be multiple classes that the image can be labeled as, or just one. If there is a single class, the term "recognition" is often applied, whereas a multi-class recognition task is often called "classification".

A subset of image classification is object detection, where specific instances of objects are identified as belonging to a certain class like animals, cars, or people.


Feature Extraction

In order to carry out image recognition/classification, the neural network must carry out feature extraction. Features are the elements of the data that you care about which will be fed through the network. In the specific case of image recognition, the features are the groups of pixels, like edges and points, of an object that the network will analyze for patterns.

Feature recognition (or feature extraction) is the process of pulling the relevant features out from an input image so that these features can be analyzed. Many images contain annotations or metadata about the image that helps the network find the relevant features.
reference https://stackabuse.com/

The first thing we should do is import the necessary libraries. I'll show how these imports are used as we go, but for now know that we'll be making use of Numpy, and various modules associated with Keras:

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [3]:
# Set random seed for purposes of reproducibility
seed = 21

In [4]:
from keras.datasets import cifar10

In [5]:
# loading in the data
     
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In most cases you will need to do some preprocessing of your data to get it ready for use, but since we are using a prepackaged dataset, very little preprocessing needs to be done. One thing we want to do is normalize the input data.

If the values of the input data are in too wide a range it can negatively impact how the network performs. In this case, the input values are the pixels in the image, which have a value between 0 to 255.

So in order to normalize the data we can simply divide the image values by 255. To do this we first need to make the data a float type, since they are currently integers. We can do this by using the astype() Numpy command and then declaring what data type we want:

In [6]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

Another thing we'll need to do to get the data ready for the network is to one-hot encode the values. I won't go into the specifics of one-hot encoding here, but for now know that the images can't be used by the network as they are, they need to be encoded first and one-hot encoding is best used when doing binary classification.

We are effectively doing binary classification here because an image either belongs to one class or it doesn't, it can't fall somewhere in-between. The Numpy command to_categorical() is used to one-hot encode. This is why we imported the np_utils function from Keras, as it contains to_categorical().

We also need to specify the number of classes that are in the dataset, so we know how many neurons to compress the final layer down to:

In [7]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

Create the Model

In [9]:
model = Sequential()

The first layer of our model is a convolutional layer. It will take in the inputs and run convolutional filters on them.

When implementing these in Keras, we have to specify the number of channels/filters we want (that's the 32 below), the size of the filter we want (3 x 3 in this case), the input shape (when creating the first layer) and the activation and padding we need.

As mentioned, relu is the most common activation, and padding='same' just means we aren't changing the size of the image at all:

In [10]:
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

ou can also string the activations and poolings together, like this:

In [11]:
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))

Now we will make a dropout layer to prevent overfitting, which functions by randomly eliminating some of the connections between the layers (0.2 means it drops 20% of the existing connections):

In [12]:
model.add(Dropout(0.2))

We may also want to do batch normalization here. Batch Normalization normalizes the inputs heading into the next layer, ensuring that the network always creates activations with the same distribution that we desire:

In [14]:
model.add(BatchNormalization())

Now comes another convolutional layer, but the filter size increases so the network can learn more complex representations:

In [16]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))


Here's the pooling layer, as discussed before this helps make the image classifier more robust so it can learn relevant patterns. There's also the dropout and batch normalization:

In [17]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

You can vary the exact number of convolutional layers you have to your liking, though each one adds more computation expenses. Notice that as you add convolutional layers you typically increase their number of filters so the model can learn more complex representations. If the numbers chosen for these layers seems somewhat arbitrary, just know that in general, you increase filters as you go on and it's advised to make them powers of 2 which can grant a slight benefit when training on a GPU.

It's important not to have too many pooling layers, as each pooling discards some data. Pooling too often will lead to there being almost nothing for the densely connected layers to learn about when the data reaches them.

The exact number of pooling layers you should use will vary depending on the task you are doing, and it's something you'll get a feel for over time. Since the images are so small here already we won't pool more than twice.

You can now repeat these layers to give your network more representations to work off of:

In [18]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

After we are done with the convolutional layers, we need to Flatten the data, which is why we imported the function above. We'll also add a layer of dropout again:

In [19]:
model.add(Flatten())
model.add(Dropout(0.2))

Now we make use of the Dense import and create the first densely connected layer. We need to specify the number of neurons in the dense layer. Note that the numbers of neurons in succeeding layers decreases, eventually approaching the same number of neurons as there are classes in the dataset (in this case 10). The kernel constraint can regularize the data as it learns, another thing that helps prevent overfitting. This is why we imported maxnorm earlier.

In [20]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In this final layer, we pass in the number of classes for the number of neurons. Each neuron represents a class, and the output of this layer will be a 10 neuron vector with each neuron storing some probability that the image in question belongs to the class it represents.

Finally, the softmax activation function selects the neuron with the highest probability as its output, voting that the image belongs to that class:

In [22]:
model.add(Dense(class_num))
model.add(Activation('softmax'))

Now that we've designed the model we want to use, we just have to compile it. Let's specify the number of epochs we want to train for, as well as the optimizer we want to use.

The optimizer is what will tune the weights in your network to approach the point of lowest loss. The Adam algorithm is one of the most commonly used optimizers because it gives great performance on most problems:

In [23]:
epochs = 25
optimizer = 'adam'

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

We can print out the model summary to see what the whole model looks like.

In [25]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        1

Now we get to training the model. To do this, all we have to do is call the fit() function on the model and pass in the chosen parameters.

Here's where I use the seed I chose, for the purposes of reproducibility.

In [ ]:
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Epoch 1/25
782/782 [==============================] - 472s 602ms/step - loss: 2.0987 - accuracy: 0.2682 - val_loss: 1.7905 - val_accuracy: 0.4193
Epoch 2/25
782/782 [==============================] - 468s 599ms/step - loss: 1.7316 - accuracy: 0.4338 - val_loss: 1.4961 - val_accuracy: 0.5085
Epoch 3/25
782/782 [==============================] - 466s 596ms/step - loss: 1.4562 - accuracy: 0.5180 - val_loss: 1.3253 - val_accuracy: 0.5453
Epoch 4/25
782/782 [==============================] - 464s 593ms/step - loss: 1.2725 - accuracy: 0.5605 - val_loss: 1.1260 - val_accuracy: 0.6287
Epoch 5/25
782/782 [==============================] - 463s 593ms/step - loss: 1.1167 - accuracy: 0.6278 - val_loss: 1.1239 - val_accuracy: 0.6279
Epoch 6/25
782/782 [==============================] - 460s 588ms/step - loss: 1.0028 - accuracy: 0.6586 - val_loss: 0.9267 - val_accuracy: 0.6956
Epoch 7/25
782/782 [==============================] - 459s 587ms/step - loss: 0.9243 - accuracy: 0.6866 - val_loss: 0.9757 -

Now we can evaluate the model and see how it performed. Just call model.evaluate()

In [ ]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))